In [5]:
import numpy as np
import depthai as dai
import cv2 as cv
import glob
from pathlib import Path
import time


In [6]:
src = 'right'

pipeline = dai.Pipeline()

cam = pipeline.createMonoCamera()

cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
cam.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

xout = pipeline.createXLinkOut()
xout.setStreamName(src)
cam.out.link(xout.input)

# Connect and start the pipeline
with dai.Device(pipeline) as device:

    q = device.getOutputQueue(name=src, maxSize=4, blocking=False)

    Path(f"right_images").mkdir(parents=True, exist_ok=True)

    for i in range(14):
        # Blocking call, will wait until a new data has arrived
        inSrc = q.get()  
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        frame = inSrc.getCvFrame()
        # Frame is transformed and ready to be shown
        cv.imshow(src, frame)

        cv.imwrite(f"right_images/{int(time.time() * 10000)}.png", frame)
        cv.waitKey(500)  

        cv.destroyAllWindows() 

In [ ]:

src = 'right'
right_images = glob.glob('right_images/*.png')

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*5,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

notFound = []

#     img_size = () # will be using this for caliberation

for fname in right_images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    cv.imshow('gray', img)        
    cv.waitKey(1000)
    cv.destroyAllWindows()

    # Finding chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,5), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,5), corners2, ret)
        cv.imshow('img', img)

        # Saving diplayed corners for future references
        cv.imwrite(f"{fname.split('.')[0]}_corners.png", img)
        cv.waitKey(1000)
        cv.destroyAllWindows()
    else :
        # if corners not found, storing it into a list

        notFound.append(fname)
        print(f"corners not found for {fname}")

cv.destroyAllWindows()

# removing the pictures whose corners werent found
# from the main image list
for i in notFound:
    images.remove(i)

# calibration
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
for fname in images:
    img = cv.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    
    # save new image into file
    cv.imwrite(f"{fname.split('.')[0]}_result.png", dst)
    
    
# we will be storing the camera matrix and 
# distortion coefficients for future uses

print("Saving camera matrix...")
camera_matrix = np.matrix(mtx)
with open(f"right_images/camera_matrix.txt",'wb') as f:
    for line in camera_matrix:
        np.savetxt(f, line, fmt='%.5f')
        
print("Saving distortion vector...")
distortion_vector = np.matrix(dist)
with open(f"right_images/distortion_matrix.txt",'wb') as f:
    for line in distortion_vector:
        np.savetxt(f, line, fmt='%.5f')
        

print("Saving rotational vectors ...")
rotation_vectors = np.array(rvecs)
with open(f"right_images/rotat_vector.txt",'wb') as f:
    for vector in rotation_vectors:
        vector = np.reshape(vector, (1,3))
        np.savetxt(f, vector, fmt='%.5f')
        
print("Saving translation vectors...")
translation_vectors = np.array(tvecs)
with open(f"right_images/trans_vector.txt",'wb') as f:
    for vector in translation_vectors:
        vector = np.reshape(vector, (1,3))
        np.savetxt(f, vector, fmt='%.5f')
        
print('Done!!')
